In [1]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.13.1+cu117 True
3.0.0
11.7
GCC 9.3


In [2]:
# Check Pytorch installation
import torch, torchvision
print("torch version:",torch.__version__, "cuda:",torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print("mmdetection:",mmdet.__version__)

# Check mmcv installation
import mmcv
print("mmcv:",mmcv.__version__)

# Check mmengine installation
import mmengine
print("mmengine:",mmengine.__version__)

torch version: 1.13.1+cu117 cuda: True
mmdetection: 3.0.0
mmcv: 2.0.0
mmengine: 0.7.3


In [3]:
import mmcv
import mmengine
import mmdet

from mmcv.transforms import Compose

from mmengine import Config
from mmengine.hooks import Hook
from mmengine.runner import set_random_seed
from mmengine.runner import Runner
from mmengine.utils import track_iter_progress

from mmdet.apis import init_detector, inference_detector
from mmdet.utils import register_all_modules
from mmdet.registry import VISUALIZERS, HOOKS

In [4]:
import os
import re
import shutil

import wandb

import numpy as np
import cv2

from pycocotools.coco import COCO

from moviepy.editor import VideoFileClip

from PIL import Image
from skimage.color import label2rgb
from skimage import io

from torchmetrics.classification import BinaryJaccardIndex

from tqdm import tqdm

import matplotlib.pyplot as plt

In [5]:
# Config file from Pre-Trained model
# https://github.com/open-mmlab/mmdetection/tree/master/configs/mask_rcnn
config = "./mmdetection/configs/mask_rcnn/mask-rcnn_r50_fpn_ms-poly-3x_coco.py"

cfg = Config.fromfile(config)

In [6]:
!mim download mmdet --config mask-rcnn_r50_fpn_mstrain-poly_3x_coco --dest ./pre_trained_checkpoints

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
processing mask-rcnn_r50_fpn_mstrain-poly_3x_coco...
mask_rcnn_r50_fpn_mstrain-poly_3x_coco_20210524_201154-21b550bb.pth exists in /home/ubuntu/pre_trained_checkpoints
Successfully dumped mask-rcnn_r50_fpn_mstrain-poly_3x_coco.py to /home/ubuntu/pre_trained_checkpoints


In [7]:
# Modify dataset classes and color
# https://github.com/open-mmlab/mmdetection/issues/9610#issuecomment-1383553783
# Change the Keys to all lower case
cfg.metainfo = {
    'classes': ('Artery', ),
    'palette': [
        (220, 20, 60),
    ]
}


albu_train_transforms = [
    dict(
        type='ShiftScaleRotate',
        shift_limit=0.3,
        scale_limit=0.2,
        rotate_limit=30,
        interpolation=1,
        p=0.4),
    dict(
        type='OneOf',
        transforms=[
            dict(
                type='CLAHE',
                clip_limit=5.0,
                tile_grid_size=(11,11),
                p=1.0),
            dict(type='ColorJitter', brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2,p=1.0),
            dict(type='RandomBrightnessContrast', brightness_limit=0.4, contrast_limit=0.4, p=1.0),
            dict(type='Equalize', p=1.0),
        ],
        p=0.2),
    dict(type='JpegCompression', quality_lower=65, quality_upper=95, p=0.1),
    dict(
        type='OneOf',
        transforms=[
            dict(type='Blur', blur_limit=3, p=1.0),
            dict(type='MedianBlur', blur_limit=5, p=1.0),
        ],
        p=0.2),
    dict(
        type='OneOf',
        transforms=[
            dict(type='ElasticTransform', p=1, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
            dict(type='GridDistortion', p=1),
            dict(type='OpticalDistortion', distort_limit=2, shift_limit=0.5, p=1)
        ],
        p=0.2),
    dict(
        type='OneOf',
        transforms=[
            dict(type='CenterCrop', height = 448, width = 448, p=1),
            dict(type='CropNonEmptyMaskIfExists', height = 720, width = 720, p=1),
            dict(type='BBoxSafeRandomCrop', erosion_rate=0.1, p=1),
        ],
        p=0.3),
]

train_pipeline = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(
        type='LoadAnnotations',
        with_bbox=True,
        with_mask=True),
        # poly2mask=True),
    dict(type='Resize', scale=(900, 720)),
    dict(
        type='Albu',
        transforms=albu_train_transforms,
        bbox_params=dict(
            type='BboxParams',
            format='pascal_voc',
            label_fields=['gt_bboxes_labels', 'gt_ignore_flags'],
            min_visibility=0.0,
            filter_lost_elements=True),
        keymap={
            'img': 'image',
            'gt_masks': 'masks',
            'gt_bboxes': 'bboxes'
        },
        skip_img_without_anno=True),
    # dict(type='RandomFlip', prob=0.5),
    dict(type='PackDetInputs', 
         meta_keys=('img_id', 'img_path', 'ori_shape', 'img_shape'))
]
  
cfg.train_pipeline = train_pipeline

In [8]:
# Single GPU
cfg.optim_wrapper.optimizer.lr = 0.2/8

cfg.train_cfg.max_epochs = 25

# Changing LR Scheduler
cfg.param_scheduler[1].end = cfg.train_cfg.max_epochs
cfg.param_scheduler[1].milestones = [10,15]

cfg.default_hooks.logger.interval = 100

In [9]:
# cfg.vis_backends = [
#         dict(type='LocalVisBackend'),
#         dict(type='TensorboardVisBackend'),
#         dict(type='WandbVisBackend', init_kwargs = dict(project='Binary_RLLPA_Aug', name=''))
#     ]
# cfg.visualizer.vis_backends = cfg.vis_backends

In [10]:
# build the runner from config
# runner = Runner.from_cfg(cfg)

In [11]:
@HOOKS.register_module()
class FindIoU(Hook):
    def __init__(self, name):
        os.makedirs("bestepochs", exist_ok=True)
        self.bestIoU = 0
        self.bestepoch = None
        self.name = name
        self.metric = BinaryJaccardIndex()
        
        # RGB format
        self.CLS2COLOR = {
            1: (228,0,120), # Red
            2: (42, 82, 190), # Blue
            3: (3, 192, 60) # Green
        }
        
        # define our custom x axis metric
        wandb.define_metric("coco/epoch")
        # define which metrics will be plotted against it
        wandb.define_metric(
          "coco/pGen1IoU", step_metric="coco/epoch", step_sync=False)
        wandb.define_metric(
          "coco/pGen2IoU", step_metric="coco/epoch", step_sync=False)
        wandb.define_metric(
          "coco/meanIoU", step_metric="coco/epoch", step_sync=False)
        
        self.artifact = wandb.Artifact(self.name, type='model')
        
    def after_val(self, runner, **kwargs):
        IoUs = []
        
        checkpoint_file = runner.work_dir + f"/epoch_{runner.epoch}.pth"
        model = init_detector(runner.cfg, checkpoint_file, device='cuda:0')
        meanIoU = []
        val_file = runner.cfg.val_dataloader.dataset.ann_file
        test_file = runner.cfg.test_dataloader.dataset.ann_file
        for f_type, json_path in zip(['pGen1', 'pGen2'], [val_file, test_file]):
            
            # json_path = f"{data_type}.json"
            coco = COCO(json_path)
            img_dir = f"combined_data"
            cat_ids = coco.getCatIds()
            frames = {}
            for idx, img_data in coco.imgs.items():
                anns_ids = coco.getAnnIds(imgIds=img_data['id'], catIds=cat_ids, iscrowd=None)
                anns = coco.loadAnns(anns_ids)

                truth_mask = coco.annToMask(anns[0])
                for i in range(1,len(anns)):
                    # mask += coco.annToMask(anns[i])
                    # mask = np.maximum(mask,coco.annToMask(anns[i])*255) # To have pixel value as 255
                    truth_mask = np.maximum(truth_mask,coco.annToMask(anns[i])*1)

                img = f'combined_data/{img_data["file_name"]}'  # or img = mmcv.imread(img), which will only load it once
                result = inference_detector(model, img)
                # outputs = predictor(im)

                pred_mask = np.zeros_like(truth_mask)
                for i in result.pred_instances.masks.type(torch.int8):
                    pred_mask = np.maximum(pred_mask, i.to('cpu').numpy().astype(np.uint8))
                    
                # frame = label2rgb(pred_mask, cv2.imread(img), alpha=0.3, bg_label=0)*255
    
                target = torch.tensor(truth_mask)
                preds = torch.tensor(pred_mask)
            
                intersection_mask = np.logical_and(pred_mask == 1, truth_mask == 1)
                pred_mask[truth_mask == 1] = 2
                pred_mask[intersection_mask] = 3
                # Repeating Channels to make it three channels
                pred_mask = np.tile(pred_mask[..., np.newaxis], (1,1,3))
                
                # red -> Wrong Predicted, blue -> Ground Truth, green -> Correct Predicted
                frame = io.imread(img)
                for color_id in range(1,4):
                    mask = np.where(pred_mask == (color_id,)*3, self.CLS2COLOR[color_id], 0).astype('uint8')
                    frame = cv2.addWeighted(frame, 1.0, mask, 0.5, 0)
                
                frames[img_data["file_name"]] = frame

                IoUs.append(self.metric(preds, target).item())
                
            
            size1,size2,_ = frame.shape
            out = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 1, (size2, size1), True)
            # Sorting the frames according to frame number eg: p3_frame_000530..PNG
            for _,i in sorted(frames.items(), key=lambda x: x[0]):
                out_img = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
                out.write(out_img)
            out.release()
            
            # Convert MPV4 codec to libx264 codec
            input_file = 'output.mp4'
            output_file = f_type+'.mp4'
            clip = VideoFileClip(input_file)
            clip.write_videofile(output_file, codec='libx264')

            # Collect all meanIoUs for all Generalization Patients
            meanIoU.append(sum(IoUs)/len(IoUs))
            print(f"IoU: {sum(IoUs)/len(IoUs)}")
            
            # axes are (time, channel, height, width)
            wandb.log({f"{self.name}_{f_type}_epoch_{runner.epoch}": wandb.Video(output_file)})
            
        for IoU, log in zip(meanIoU, ['pGen1', 'pGen2']):
            wandb.log({f'coco/{log}':IoU, 'coco/epoch':runner.epoch})
            
        meanIoU = sum(meanIoU)/len(meanIoU)
        if meanIoU > self.bestIoU:
            self.bestIoU = meanIoU
            self.bestepoch = checkpoint_file

        print(f"meanIoU: {meanIoU}")
        wandb.log({'coco/iou':meanIoU, 'coco/epoch':runner.epoch})
        
        print(f"Saving checkpoint of epoch {runner.epoch} to wandb")
        self.artifact.add_file(checkpoint_file, name=f'epoch_{runner.epoch}.pth')
        # wandb.log_artifact(self.artifact)
    def after_run(self,runner, **kwargs):
        shutil.copy(self.bestepoch, f"bestepochs/{self.name}.pth")
        print(f"Saving best checkpoint to wandb")
        self.artifact.add_file(self.bestepoch, name=f"best.pth")
        wandb.log_artifact(self.artifact)

In [ ]:
for file_name in os.listdir("./JSONS"):
    # If file doesn't contain "AllPatients" continue
    if "All" not in file_name:
        continue
    train_file = file_name
    pGen_names = file_name.split('-')[1]
    first_p = pGen_names.find('p')
    second_p = pGen_names.rfind('p')
    val_file = pGen_names[first_p:second_p]+".json"
    test_file = pGen_names[second_p:]
    
    # train_file = val_file = test_file = 'p10.json'
    
    if val_file in ['p1.json', 'p3.json', 'p5.json']:
        continue

    name = file_name.split('.')[0]
    
    cfg.custom_hooks = [
        dict(type='FindIoU', name=name)
    ]
    
    cfg.vis_backends = [
        dict(type='LocalVisBackend'),
        dict(type='TensorboardVisBackend'),
        dict(type='WandbVisBackend', init_kwargs = dict(project='Binary_RLLPA_Aug', name=name))
    ]
    cfg.visualizer.vis_backends = cfg.vis_backends

    cfg.data_root = '.'

    cfg.train_dataloader.batch_size = 12
    cfg.train_dataloader.num_workers = 24
    cfg.train_dataloader.dataset.dataset.ann_file = f'./JSONS/{train_file}'
    cfg.train_dataloader.dataset.dataset.data_root = cfg.data_root
    cfg.train_dataloader.dataset.dataset.data_prefix.img = 'combined_data/'
    cfg.train_dataloader.dataset.dataset.metainfo = cfg.metainfo
    
    cfg.train_dataloader.dataset.dataset.pipeline = cfg.train_pipeline
    
    cfg.train_dataloader.dataset.times = 2

    cfg.val_dataloader.batch_size = 12
    cfg.val_dataloader.num_workers = 24
    cfg.val_dataloader.dataset.ann_file = f'./JSONS/{val_file}'
    cfg.val_dataloader.dataset.data_root = cfg.data_root
    cfg.val_dataloader.dataset.data_prefix.img = 'combined_data/'
    cfg.val_dataloader.dataset.metainfo = cfg.metainfo

    cfg.test_dataloader.batch_size = 12
    cfg.test_dataloader.dataset.ann_file = f'./JSONS/{test_file}'
    cfg.test_dataloader.dataset.data_root = cfg.data_root
    cfg.test_dataloader.dataset.data_prefix.img = 'combined_data/'
    cfg.test_dataloader.dataset.metainfo = cfg.metainfo

    # Modify metric config
    cfg.val_evaluator.ann_file = cfg.data_root+'/'+f'JSONS/{val_file}'
    cfg.test_evaluator.ann_file = cfg.data_root+'/'+f'JSONS/{test_file}'

    # Modify num classes of the model in box head and mask head
    cfg.model.roi_head.bbox_head.num_classes = 1
    cfg.model.roi_head.mask_head.num_classes = 1

    # We can still the pre-trained Mask RCNN model to obtain a higher performance
    cfg.load_from = './pre_trained_checkpoints/mask_rcnn_r50_fpn_mstrain-poly_3x_coco_20210524_201154-21b550bb.pth'

    # Set up working dir to save files and logs.
    cfg.work_dir = f"./Binary_RLLPA_Aug/{name}"
    
    try:
        runner = Runner.from_cfg(cfg)
    except wandb.Error:
        wandb.init(dir=cfg.work_dir, project='Binary_RLLPA_Aug', name=name)
        runner = Runner.from_cfg(cfg)

    runner.train()

05/25 16:37:57 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.8.10 (default, Nov 14 2022, 12:59:47) [GCC 9.4.0]
    CUDA available: True
    numpy_random_seed: 517293159
    GPU 0: NVIDIA A10
    CUDA_HOME: /usr
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: x86_64-linux-gnu-gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
    PyTorch: 1.13.1+cu117
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.7
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;

wandb: Currently logged in as: skhader (plakhsa-mgh). Use `wandb login --relogin` to force relogin


05/25 16:38:04 - mmengine - INFO - Distributed training is not used, all SyncBatchNorm (SyncBN) layers in the model will be automatically reverted to BatchNormXd layers if they are used.
05/25 16:38:04 - mmengine - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) RuntimeInfoHook                    
(BELOW_NORMAL) LoggerHook                         
 -------------------- 
before_train:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(VERY_LOW    ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(NORMAL      ) DistSamplerSeedHook                
 -------------------- 
before_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
 -------------------- 
after_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                

Moviepy - Done !
Moviepy - video ready pGen1.mp4
IoU: 0.21404794042203332
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Moviepy - Building video pGen2.mp4.
Moviepy - Writing video pGen2.mp4



Moviepy - Done !
Moviepy - video ready pGen2.mp4
IoU: 0.3362414434629087
meanIoU: 0.275144691942471
Saving checkpoint of epoch 1 to wandb
05/25 16:52:51 - mmengine - INFO - Epoch(train)  [2][100/465]  lr: 2.5000e-02  eta: 3:18:17  time: 1.0982  data_time: 0.0210  memory: 16560  loss: 0.6488  loss_rpn_cls: 0.0116  loss_rpn_bbox: 0.0185  loss_cls: 0.1342  acc: 95.2799  loss_bbox: 0.2676  loss_mask: 0.2169
05/25 16:54:41 - mmengine - INFO - Epoch(train)  [2][200/465]  lr: 2.5000e-02  eta: 3:17:15  time: 1.0976  data_time: 0.0127  memory: 16547  loss: 0.6410  loss_rpn_cls: 0.0092  loss_rpn_bbox: 0.0182  loss_cls: 0.1268  acc: 96.0286  loss_bbox: 0.2711  loss_mask: 0.2157
05/25 16:56:31 - mmengine - INFO - Epoch(train)  [2][300/465]  lr: 2.5000e-02  eta: 3:15:47  time: 1.0946  data_time: 0.0125  memory: 16639  loss: 0.5894  loss_rpn_cls: 0.0089  loss_rpn_bbox: 0.0165  loss_cls: 0.1156  acc: 94.9544  loss_bbox: 0.2465  loss_mask: 0.2018
05/25 16:58:21 - mmengine - INFO - Epoch(train)  [2][40

Moviepy - Done !
Moviepy - video ready pGen1.mp4
IoU: 0.17772409696199504
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Moviepy - Building video pGen2.mp4.
Moviepy - Writing video pGen2.mp4



Moviepy - Done !
Moviepy - video ready pGen2.mp4
IoU: 0.2696863304348013
meanIoU: 0.22370521369839816
Saving checkpoint of epoch 2 to wandb


In [ ]:
# # load tensorboard in colab
# %load_ext tensorboard

# # see curves in tensorboard
# %tensorboard --logdir ./Binary_RLLPA_Aug --bind_all

In [ ]:
# Zip the logs and everything without the weights
!zip -r -qq Binary_RLLPA_Aug.zip Binary_RLLPA_Aug -x \*.pth \*last_checkpoint \*.mp4
!zip -r -qq bestepochs_Binary_RLLPA_Aug.zip bestepochs

In [ ]:
!aws s3 cp bestepochs_Binary_RLLPA_Aug.zip s3://all-patients/
!aws s3 cp Binary_RLLPA_Aug.zip s3://all-patients/

In [ ]:
print("DONE and Moved")